In [1]:
# STEP 1: Import libraries
from prophet import Prophet
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# STEP 2: Set stock and date range
ticker = 'TSLA'
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=5*365)).strftime('%Y-%m-%d')

# STEP 3: Download stock data from Yahoo Finance
df = yf.download(ticker, start=start_date, end=end_date)
print("Raw data from yf.download:")
print(df.head())  # Check raw output
print("Columns available:", df.columns)  # Verify column names

# STEP 4: Prepare data
# Select only 'Date' and 'Close', ensuring 'Close' is a Series
df = df.reset_index()[['Date', 'Close']]
df.columns = ['ds', 'y']  # Rename directly to 'ds' and 'y'
df['ds'] = pd.to_datetime(df['ds'], errors='coerce')  # Force datetime
df['y'] = pd.to_numeric(df['y'], errors='coerce')     # Ensure y is numeric, no squeeze needed
df = df.fillna(method='ffill')  # Fill missing values

# Debug
print("DataFrame dtypes after conversion:")
print(df.dtypes)  # Should show ds: datetime64[ns], y: float64
print("First few rows of df:")
print(df.head())  # Verify data

# Check for NaN values or empty DataFrame
if df.empty:
    raise ValueError("Downloaded DataFrame is empty. Check ticker or date range.")
if df['ds'].isnull().any() or df['y'].isnull().any():
    raise ValueError("NaN values found in 'ds' or 'y' after conversion")

# STEP 5: Fit Prophet model
model = Prophet(
    changepoint_prior_scale=0.05,
    weekly_seasonality=True,
    yearly_seasonality=True
)
model.add_country_holidays('US')
model.fit(df)

# STEP 6: Forecast the next 30 days
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

# STEP 7: Extract last 30 days of forecast, filter to weekdays
forecast_out = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(30)
forecast_out = forecast_out[forecast_out['ds'].dt.dayofweek < 5]

# STEP 8: Export to CSV for Power BI
forecast_out.to_csv("stock_forecast.csv", index=False)

# STEP 9: (Optional) Show preview
print("Preview of forecast:")
print(forecast_out.head())

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
C:\Users\DELL\AppData\Local\Temp\ipykernel_20108\2184363914.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')  # Fill missing values


Raw data from yf.download:
Price           Close       High        Low       Open     Volume
Ticker           TSLA       TSLA       TSLA       TSLA       TSLA
Date                                                             
2020-04-09  38.200001  38.345333  37.140667  37.472668  204750000
2020-04-13  43.396667  43.466667  38.702000  39.344002  337131000
2020-04-14  47.326000  49.458668  46.161999  46.598000  458647500
2020-04-15  48.655334  50.208668  47.333332  49.466667  353655000
2020-04-16  49.680668  50.630001  47.114666  47.796001  309868500
Columns available: MultiIndex([( 'Close', 'TSLA'),
            (  'High', 'TSLA'),
            (   'Low', 'TSLA'),
            (  'Open', 'TSLA'),
            ('Volume', 'TSLA')],
           names=['Price', 'Ticker'])
DataFrame dtypes after conversion:
ds    datetime64[ns]
y            float64
dtype: object
First few rows of df:
          ds          y
0 2020-04-09  38.200001
1 2020-04-13  43.396667
2 2020-04-14  47.326000
3 2020-04-15  48.6

20:06:45 - cmdstanpy - INFO - Chain [1] start processing
20:06:45 - cmdstanpy - INFO - Chain [1] done processing


Preview of forecast:
             ds        yhat  yhat_lower  yhat_upper
1255 2025-04-08  334.734537  289.266572  377.768676
1256 2025-04-09  334.652747  290.489334  378.289313
1257 2025-04-10  334.124132  288.988956  378.312718
1258 2025-04-11  334.502736  287.160053  379.180060
1261 2025-04-14  335.914708  292.067214  379.881449


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import time

# Define the stock ticker
ticker_symbol = "TSLA"  # Change as needed

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Function to get the latest price
def get_latest_price():
    live_data = ticker.history(period="1d", interval="1m")  # 1-minute interval for near real-time
    latest_price = live_data['Close'].iloc[-1]
    latest_time = live_data.index[-1]
    return pd.DataFrame({
        "Ticker": [ticker_symbol],
        "Price": [latest_price],
        "Timestamp": [latest_time]
    })

# Test the function
data = get_latest_price()
print(data)

# Save the latest data to a CSV file
data.to_csv("live_stock_price.csv", index=False)
print("Data saved to live_stock_price.csv")


# Loop to update the CSV every 10 seconds
while True:
    data = get_latest_price()
    data.to_csv("live_stock_price.csv", index=False)
    print(f"Updated at {datetime.now()}: {data['Price'].iloc[0]}")
    time.sleep(10)  # Update every 10 seconds

  Ticker       Price                 Timestamp
0   TSLA  249.605896 2025-04-08 10:36:00-04:00
Data saved to live_stock_price.csv
Updated at 2025-04-08 20:06:46.550801: 249.60589599609375
Updated at 2025-04-08 20:06:56.634525: 249.60589599609375
Updated at 2025-04-08 20:07:06.751075: 249.3800048828125
Updated at 2025-04-08 20:07:16.855546: 249.1300048828125
Updated at 2025-04-08 20:07:26.934875: 249.1300048828125
Updated at 2025-04-08 20:07:37.071938: 249.63429260253906
Updated at 2025-04-08 20:07:47.171573: 249.63429260253906
Updated at 2025-04-08 20:07:57.299360: 249.2899932861328
Updated at 2025-04-08 20:08:07.389845: 249.2899932861328
Updated at 2025-04-08 20:08:17.541580: 248.3800048828125
Updated at 2025-04-08 20:08:27.641529: 248.3800048828125
Updated at 2025-04-08 20:08:37.768711: 248.30999755859375
